In [1]:
import re
import os
import copy
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from skimage.io import imread
from sklearn.neighbors import KNeighborsClassifier as ks
from sklearn.metrics.pairwise import pairwise_kernels

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
arr=[]
labels = []
i=0
j=0
for root,di,files in os.walk("/content/drive/MyDrive/Colab Notebooks/Face-Recognition/dataset"): 
  for name in files:
    if name.endswith("pgm"):
      root_folder = os.path.split(root)
      res=re.findall('(\d+)',root_folder[1])
      image = imread(os.path.join(root,name))
      image=image.flatten(order='C')
      arr.append(image)
      labels.append(int(res[0]))
      i=i+1
arr=np.array(arr)   
labels=np.array(labels)    
arr = arr/255

In [4]:
print(labels.shape)       
print(arr.shape)
print(labels[2])

(400,)
(400, 10304)
1


In [5]:
# adding the data to a dataframe
df = pd.DataFrame(arr) 
df['label'] = pd.DataFrame(labels)
# sorting the data according to labels
df.sort_values("label", inplace=True)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,10295,10296,10297,10298,10299,10300,10301,10302,10303,label
0,0.168627,0.196078,0.160784,0.227451,0.305882,0.325490,0.262745,0.188235,0.172549,0.180392,...,0.643137,0.643137,0.635294,0.623529,0.611765,0.607843,0.619608,0.600000,0.662745,1
8,0.247059,0.207843,0.137255,0.141176,0.129412,0.133333,0.121569,0.137255,0.152941,0.168627,...,0.662745,0.650980,0.631373,0.619608,0.662745,0.537255,0.160784,0.039216,0.094118,1
7,0.160784,0.184314,0.184314,0.180392,0.172549,0.192157,0.188235,0.227451,0.239216,0.192157,...,0.133333,0.137255,0.133333,0.141176,0.133333,0.152941,0.137255,0.145098,0.149020,1
6,0.172549,0.168627,0.125490,0.125490,0.117647,0.117647,0.149020,0.156863,0.188235,0.258824,...,0.164706,0.164706,0.156863,0.129412,0.113725,0.145098,0.168627,0.168627,0.145098,1
5,0.188235,0.192157,0.176471,0.184314,0.192157,0.223529,0.152941,0.164706,0.207843,0.192157,...,0.172549,0.156863,0.160784,0.192157,0.164706,0.172549,0.184314,0.180392,0.180392,1


In [6]:
# seperating images from labels
Y = np.array(df['label'])
df = df.drop(axis=1, labels="label")

In [7]:
df = df.reset_index()
df.head()

,index,0,1,2,3,4,5,6,7,8,...,10294,10295,10296,10297,10298,10299,10300,10301,10302,10303
0,0,0.168627,0.196078,0.160784,0.227451,0.305882,0.325490,0.262745,0.188235,0.172549,...,0.654902,0.643137,0.643137,0.635294,0.623529,0.611765,0.607843,0.619608,0.600000,0.662745
1,8,0.247059,0.207843,0.137255,0.141176,0.129412,0.133333,0.121569,0.137255,0.152941,...,0.678431,0.662745,0.650980,0.631373,0.619608,0.662745,0.537255,0.160784,0.039216,0.094118
2,7,0.160784,0.184314,0.184314,0.180392,0.172549,0.192157,0.188235,0.227451,0.239216,...,0.105882,0.133333,0.137255,0.133333,0.141176,0.133333,0.152941,0.137255,0.145098,0.149020
3,6,0.172549,0.168627,0.125490,0.125490,0.117647,0.117647,0.149020,0.156863,0.188235,...,0.145098,0.164706,0.164706,0.156863,0.129412,0.113725,0.145098,0.168627,0.168627,0.145098
4,5,0.188235,0.192157,0.176471,0.184314,0.192157,0.223529,0.152941,0.164706,0.207843,...,0.152941,0.172549,0.156863,0.160784,0.192157,0.164706,0.172549,0.184314,0.180392,0.180392


In [8]:
df = df.drop(axis=1, labels="index")
df.head()

,0,1,2,3,4,5,6,7,8,9,...,10294,10295,10296,10297,10298,10299,10300,10301,10302,10303
0,0.168627,0.196078,0.160784,0.227451,0.305882,0.325490,0.262745,0.188235,0.172549,0.180392,...,0.654902,0.643137,0.643137,0.635294,0.623529,0.611765,0.607843,0.619608,0.600000,0.662745
1,0.247059,0.207843,0.137255,0.141176,0.129412,0.133333,0.121569,0.137255,0.152941,0.168627,...,0.678431,0.662745,0.650980,0.631373,0.619608,0.662745,0.537255,0.160784,0.039216,0.094118
2,0.160784,0.184314,0.184314,0.180392,0.172549,0.192157,0.188235,0.227451,0.239216,0.192157,...,0.105882,0.133333,0.137255,0.133333,0.141176,0.133333,0.152941,0.137255,0.145098,0.149020
3,0.172549,0.168627,0.125490,0.125490,0.117647,0.117647,0.149020,0.156863,0.188235,0.258824,...,0.145098,0.164706,0.164706,0.156863,0.129412,0.113725,0.145098,0.168627,0.168627,0.145098
4,0.188235,0.192157,0.176471,0.184314,0.192157,0.223529,0.152941,0.164706,0.207843,0.192157,...,0.152941,0.172549,0.156863,0.160784,0.192157,0.164706,0.172549,0.184314,0.180392,0.180392


In [9]:
X_train = []
X_test =  []  
labels_train = []
labels_test =  []  

for i in range(0,400):
    if((i%2==0)):
       X_test.append(arr[i,:])
       labels_test.append(labels[i])
    else:
       X_train.append(arr[i,:])
       labels_train.append(labels[i])
 
print(X_test[0])
print(labels_test[0])

[0.16862745 0.19607843 0.16078431 ... 0.61960784 0.6        0.6627451 ]
1


In [10]:
X_train = np.array(X_train)
X_test =  np.array(X_test)
labels_train = np.array(labels_train)
labels_test =  np.array(labels_test)

In [11]:
def get_kernel(X):
  # calculate the mean
  kernel = pairwise_kernels(X.T)
  # center the data
  n = kernel.shape[0]
  eye = np.eye(n)
  one = np.ones((n,n))
  kmat = eye - (1/n)*one
  kernel = np.matmul(np.matmul(kmat,kernel),kmat)
  return kernel

In [12]:
def KPCA(X):
  # X           ---> data to be reduced
  # alpha       ---> total variance threshold
  # evectors    ---> eigen vectors used to reduce the dimensionality

  kernel = get_kernel(X)
  # calculate eigenvalues and eigenvectors
  (evalues, evectors) = np.linalg.eigh(kernel)
  evalues = np.flip(evalues)
  evectors = np.flip(evectors,axis=1)

  return (evalues, evectors , kernel)

In [13]:
def choose_dim(evalues, evectors, alpha):
  # fraction of total variance
  fr = evalues/evalues.sum()

  totvar = 0
  i = 0

  while(totvar < alpha):
    totvar = totvar + fr[i]
    i = i + 1

  # choose dimensionality
  evectors = evectors[ : , 0:i]

  return evectors

In [14]:
def reduce_dim(X, evectors):
  reducedmat = np.matmul(evectors.T,X.T).T
  return reducedmat

In [15]:
# DO NOT RUN THIS CELL IF YOU ALREADY HAVE THE PICKLE FILES
(evalues, evectors, kernel) = KPCA(X_train)
# DO NOT RUN THIS CELL IF YOU ALREADY HAVE THE PICKLE FILES

In [16]:
print(kernel.shape)
print(evectors.shape)

(10304, 10304)
(10304, 10304)


In [37]:
evectors_red = choose_dim(evalues, evectors, 0.95)

In [38]:
print(evectors_red.shape)

(10304, 93)


In [39]:
X_train_red = reduce_dim(X_train, evectors_red)
X_test_red =  reduce_dim(X_test, evectors_red)

In [40]:
X_test_red.shape

(200, 93)

In [41]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train_red, labels_train)

KNeighborsClassifier(n_neighbors=1)

In [42]:
knn.score(X_test_red, labels_test)

0.945